In [197]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn

from scipy.io import loadmat
from torchvision.models import efficientnet_b0
from torch.nn import (
    Conv2d,
    Linear
)

from numpy_utils.layers import (
    Layer,
    LinearLayer,
    ConvLayer
)
from data_utils.dataset import CarsDataset

In [144]:
# plt.rcParams['figure.figsize'] = (15, 5)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [105]:
def annot_mat_to_csv(mat_path, csv_path):
    meta = loadmat(mat_path)
    annots = meta['annotations']
    c_name = annots[:, :][['class', 'fname']].squeeze(0)
    c = list(map(lambda x: x.item(), c_name['class']))
    name = list(map(lambda x: x.item(), c_name['fname']))
    df = pd.DataFrame({
        'class': c,
        'fname': name
    })
    df.to_csv(csv_path, index=False)

In [106]:
annot_mat_to_csv('../data/cars_train_annos.mat', '../data/train.csv')

In [112]:
dts = CarsDataset('../data/train', '../data/train.csv')

In [132]:
it = iter(dts)
hws = np.array((list(map(lambda x: x[0].size, it))))
h, w = map(lambda x: x.squeeze(1), np.split(hws, 2, axis=1))
print(h.shape, w.shape)

(8144,) (8144,)


In [135]:
print(len(np.unique(h)))
print(len(np.unique(w)))

654
829


In [185]:
lt = Linear(512, 128)
ln = LinearLayer(512, 128)

In [186]:
ln.W = lt.weight.data.numpy()
ln.b = lt.bias.data.numpy()

In [187]:
X = torch.rand((1, 512))
Xn = X.numpy()

In [196]:
out = lt(X)
outn = ln(Xn)

torch.Size([1, 128]) (1, 128)
8.324903e-13


In [425]:
ct = Conv2d(3, 5, 3)
cn = ConvLayer(3, 5, 3)
wt, bt = ct._parameters.values()
cn.W = np.transpose(wt.data.numpy(), (0, 1, 2, 3))
cn.W = wt.data.numpy()
cn.b = bt.data.numpy()

In [426]:
X = torch.rand(1, 3, 4, 4)
Xn = X.numpy()
# Xn = np.arange(48).reshape(1, 3, 4, 4)

In [427]:
out = ct(X).detach().numpy()
outn, outn2 = cn(Xn)

In [428]:
print(np.power(out - outn, 2).sum())
print(np.power(out - outn2, 2).sum())
print(np.sum(out))
print(np.sum(outn))
print(np.sum(outn2))

2.0337913
91.17760101532012
-3.7989595
-2.0108993
-25.38761341571808


In [358]:
print(np.subtract(out, outn, ))

[[ 0.         -0.24123564  0.04461378  0.13161522]
 [-0.2850863  -0.2065641   0.048316   -0.0236195 ]
 [ 0.3031059  -0.00441706  0.14700645  0.08255455]
 [-0.34693524 -0.0659239  -0.08593747 -0.34126934]]
